In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam



In [2]:
from sklearn.model_selection import train_test_split

gamedata = pd.read_csv('/Users/mgor/Downloads/game.csv', error_bad_lines=False)
gamedata = gamedata[['home_team_id','away_team_id','home_goals','away_goals']]
train, test = train_test_split(gamedata, test_size=0.1)
gamedata = train
# print(type(train))
# print(train)
# print(test)
# print(next(test.iterrows())[1])
gamedata = gamedata.rename(columns={'home_goals': 'HomeGoals', 'away_goals': 'AwayGoals'})
gamedata.head()

,home_team_id,away_team_id,HomeGoals,AwayGoals
5755,15,6,4,3
6111,13,15,2,4
6358,1,4,3,5
4428,1,2,4,3
7012,2,12,6,4


In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

goal_model_data = pd.concat([gamedata[['home_team_id','away_team_id','HomeGoals']].assign(home=1).rename(
            columns={'home_team_id':'team', 'away_team_id':'opponent','HomeGoals':'goals'}),
           gamedata[['away_team_id','home_team_id','AwayGoals']].assign(home=0).rename(
            columns={'away_team_id':'team', 'home_team_id':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                13392
Model:                            GLM   Df Residuals:                    13388
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -25062.
Date:                Sun, 18 Nov 2018   Deviance:                       14340.
Time:                        21:05:53   Pearson chi2:                 1.26e+04
No. Iterations:                     4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9588      0.012     77.121      0.000       0.934       0.983
home           0.1081      0.010     10.391      0.000       0.088       0.128
team       -2.421e-05      0.000     -0.057      0.955      -0.001       0.001
opponent       0.0003      0.000      0.682      0.495      -0.001       0.001
==============================================================================
"""

In [4]:
from sklearn.metrics import mean_squared_error

actuals_home = []
predictions_home = []
actuals_away = []
predictions_away = []
hit = 0
miss = 0
for row in test.iterrows():
#     print(row[1].home_team_id)
#     print(poisson_model.predict(pd.DataFrame(data={'team': row[1].home_team_id, 'opponent': row[1].away_team_id,
#                                        'home':1},index=[1])).values)
    actuals_home.append(row[1].home_goals)
    actuals_away.append(row[1].away_goals)
    predictions_home.append(poisson_model.predict(pd.DataFrame(data={'team': row[1].home_team_id, 'opponent': row[1].away_team_id,
                                       'home':1},index=[1])).values[0])
    predictions_away.append(poisson_model.predict(pd.DataFrame(data={'team': row[1].away_team_id, 'opponent': row[1].home_team_id,
                                       'home':0},index=[1])).values[0])
    if ((row[1].home_goals > row[1].away_goals and round(predictions_home[len(predictions_home)-1]) > round(predictions_away[len(predictions_home)-1])) or (row[1].home_goals < row[1].away_goals and round(predictions_home[len(predictions_home)-1]) < round(predictions_away[len(predictions_home)-1])) or (row[1].home_goals == row[1].away_goals and round(predictions_home[len(predictions_home)-1]) == round(predictions_away[len(predictions_home)-1]))):
        hit += 1
    else:
        miss +=1
    

print(mean_squared_error(actuals_home, predictions_home))
print(mean_squared_error(actuals_away, predictions_away))
print(hit)
print(miss)

2.631776860084123
2.6896152230297097
0
745


In [5]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

In [6]:
hit = 0
miss = 0
for row in test.iterrows():
    temp = simulate_match(poisson_model, row[1].home_team_id, row[1].away_team_id)
    home = np.sum(np.tril(temp, -1))
    draw = np.sum(np.tril(temp))
    away = np.sum(np.tril(temp, 1))
    if ((home > draw and home > away and row[1].home_goals > row[1].away_goals) or (away > home and away > draw and row[1].home_goals < row[1].away_goals) or (draw > home and draw > away and row[1].home_goals == row[1].away_goals)):
        hit += 1
    else:
        miss += 1

print(hit)
print(miss)

372
373
